# High school network and deep learning

Our point is, considering the positive and negative networks as two different directed graphs, use a GConv network with previous autoencoder (Tutorial 12 Pytorch Geometric) to predict links from one of the networks, then compare the predicted_edges with the negative ones. Consider the positive as the training set and compare it to the negatives in the test one. 

* We are going to use a graph autoencoder, which is a non-supervised neural network that takes data, translate them to another representation (the one the neural network extracts from them) and then try to rebuild the original data. The representation it learns is based on the structure of the network.

* We will use also a heuristic method, called PageRank method, traditionally used in link prediction, where the probability of a link depends on a variable called rank. 

* We compare it with a null method, a random graph. 

In both of the comparative methods of the GNN, negative links can only be placed where there are not positive links. 

The graph autoencoder generate a fixed number of links depending on built-in functions, so we are taking these number of links in order to establish comparison with other methods.

In [1]:
import pandas as pd

In [2]:
nodes = pd.read_csv("Renacimiento_info_completo_1.csv",encoding="iso8859_7",delimiter=";")
edges = pd.read_csv("Renacimiento_edges_completo_1.csv",encoding="iso8859_7",delimiter=";")

In [3]:
nodes["Curso"] = nodes["Curso"].apply(lambda x: x.split("Ί")[0])
edges.drop("relacion",axis=1,inplace=True)


In [4]:
clean_range = dict(nodes["ID"])
clean_range = {value:key for key,value in clean_range.items()}
nodes["ID"] = nodes["ID"].map(clean_range)
edges["from"] = edges["from"].map(clean_range)
edges["to"] = edges["to"].map(clean_range)

In [6]:
import networkx as nx

pos_edges = edges[edges["peso"]> 0]
neg_edges = edges[edges["peso"]< 0]
G_positive = nx.from_pandas_edgelist(pos_edges, "from", "to",create_using=nx.DiGraph,edge_attr="peso")
G_negative = nx.from_pandas_edgelist(neg_edges, "from", "to",create_using=nx.DiGraph,edge_attr="peso")
G_negative.add_nodes_from(range(nodes.index.max()+1))

### Define the complementary network of the positive network
We will use it for choosing the edges in the different null models. 

In [7]:
edges_real = []
for elem in pos_edges[["from","to"]].to_numpy():
    edges_real.append(tuple(elem))
chosen_edges = list(nx.complete_graph(238,create_using=nx.DiGraph()).edges())
for elem in edges_real:
    chosen_edges.remove(elem)


## Graph autoencoders

### Load the dataset

In [8]:
import numpy as np
import pandas as pd 


import torch
import torch_geometric.data as data
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.utils import negative_sampling,train_test_split_edges,to_dense_adj
from sklearn.metrics import roc_auc_score
from torch_geometric.transforms import RandomLinkSplit
from sklearn import preprocessing

device = "cpu"

In [9]:
### One hot encode and normalize node attributes
nodes_dummy = pd.get_dummies(nodes,['Curso', 'Grupo', 'Sexo', 'Procedencia', 'Repetidor'])
rng = np.random.default_rng()
#nodes_dummy = pd.DataFrame(rng.integers(0, 2, size=(409, 10)), columns=list('ABCDEFGHIJ'))

x = nodes_dummy.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
nodes_norm = pd.DataFrame(x_scaled)

### Firstly, check for isomorphism with Networkx 

Networkx has a isomorphism library that comes mainly from the VF2 algorithm : https://www.researchgate.net/publication/200034365_An_Improved_Algorithm_for_Matching_Large_Graphs


In [10]:
from networkx.algorithms import isomorphism


DiGM = isomorphism.DiGraphMatcher(G_positive,G_negative)

print("The graph of positive links is direcly isomorphic to the negative one ? {}.".format(DiGM.is_isomorphic()))

The graph of positive links is direcly isomorphic to the negative one ? False.


In [11]:
###Without including class and group information 
positive_data = data.Data(x=torch.tensor(nodes_norm.to_numpy(),dtype=torch.float32),
                          edge_index=torch.tensor(pos_edges[["from","to"]].to_numpy().T))
negative_data = data.Data(x=torch.tensor(nodes_norm.to_numpy(),dtype=torch.float32),
                          edge_index=torch.tensor(neg_edges[["from","to"]].to_numpy().T))

In [12]:
positive_data,negative_data

(Data(x=[238, 36], edge_index=[2, 3397]),
 Data(x=[238, 36], edge_index=[2, 358]))

In [13]:
data = positive_data.clone()
data.num_nodes = len(data._store["x"])
data = train_test_split_edges(data)


/home/juan/.local/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


### Models for the neural network

#### Autoencoder

In [14]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(data.num_features, 128)
        self.conv2 = GCNConv(128, 64)

    def encode(self):
        x = self.conv1(data.x, data.train_pos_edge_index) # convolution 1
        x = x.relu()
        return self.conv2(x, data.train_pos_edge_index) # convolution 2

    def decode(self, z, pos_edge_index, neg_edge_index): # only pos and neg edges
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) # concatenate pos and neg edges
        logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # dot product 
        return logits

    def decode_all(self, z): 
        prob_adj = z @ z.t() # get adj NxN
        return (prob_adj > 1-10e-10).nonzero(as_tuple=False).t() # get predicted edge_list 

#### Set the parameters and move data to autoencoder

In [15]:
model, positive_data = Net().to(device), positive_data.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

#### Algorithms of training and evaluation (Tutorial PyG)

In [16]:

def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train():
    model.train()

    neg_edge_index = negative_sampling(
        edge_index=data.train_pos_edge_index, #positive edges
        num_nodes=data.num_nodes, # number of nodes
        num_neg_samples=data.train_pos_edge_index.size(1)) # number of neg_sample equal to number of pos_edges

    optimizer.zero_grad()
    
    z = model.encode() #encode
    link_logits = model.decode(z, data.train_pos_edge_index, neg_edge_index) # decode
    
    link_labels = get_link_labels(data.train_pos_edge_index, neg_edge_index)
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()

    return loss


@torch.no_grad()
def test():
    model.eval()
    perfs = []
    for prefix in ["val", "test"]:
        pos_edge_index = data[f'{prefix}_pos_edge_index']
        neg_edge_index = data[f'{prefix}_neg_edge_index']

        z = model.encode() # encode train
        link_logits = model.decode(z, pos_edge_index, neg_edge_index) # decode test or val
        link_probs = link_logits.sigmoid() # apply sigmoid
        
        link_labels = get_link_labels(pos_edge_index, neg_edge_index) # get link
        
        perfs.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score
    return perfs


#### Training and test

In [17]:
best_val_perf = test_perf = 0
for epoch in range(1, 2001):
    train_loss = train()
    val_perf, tmp_test_perf = test()
    if val_perf > best_val_perf:
        best_val_perf = val_perf
        test_perf = tmp_test_perf
    log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    if epoch % 100 == 0:
        print(log.format(epoch, train_loss, best_val_perf, test_perf))

Epoch: 100, Loss: 0.3782, Val: 0.9710, Test: 0.9582
Epoch: 200, Loss: 0.3900, Val: 0.9752, Test: 0.9565
Epoch: 300, Loss: 0.3776, Val: 0.9768, Test: 0.9540
Epoch: 400, Loss: 0.3855, Val: 0.9768, Test: 0.9540
Epoch: 500, Loss: 0.3635, Val: 0.9768, Test: 0.9540
Epoch: 600, Loss: 0.3790, Val: 0.9768, Test: 0.9540
Epoch: 700, Loss: 0.3625, Val: 0.9768, Test: 0.9540
Epoch: 800, Loss: 0.3712, Val: 0.9768, Test: 0.9540
Epoch: 900, Loss: 0.3599, Val: 0.9768, Test: 0.9540
Epoch: 1000, Loss: 0.3720, Val: 0.9768, Test: 0.9540
Epoch: 1100, Loss: 0.3589, Val: 0.9768, Test: 0.9540
Epoch: 1200, Loss: 0.3738, Val: 0.9768, Test: 0.9540
Epoch: 1300, Loss: 0.3630, Val: 0.9768, Test: 0.9540
Epoch: 1400, Loss: 0.3724, Val: 0.9768, Test: 0.9540
Epoch: 1500, Loss: 0.3630, Val: 0.9768, Test: 0.9540
Epoch: 1600, Loss: 0.3559, Val: 0.9768, Test: 0.9540
Epoch: 1700, Loss: 0.3610, Val: 0.9768, Test: 0.9540
Epoch: 1800, Loss: 0.3622, Val: 0.9768, Test: 0.9540
Epoch: 1900, Loss: 0.3554, Val: 0.9768, Test: 0.9540
Ep

In [18]:
z = model.encode()
final_edge_index_1 = model.decode_all(z)
#Remove self loops
bool_mask = final_edge_index_1[0] != final_edge_index_1[1]
simulated_edges_1 = torch.empty((2,int(sum(bool_mask))))
for item in range(final_edge_index_1.size()[0]):
    simulated_edges_1[item] = final_edge_index_1[item][bool_mask]
    

In [19]:
negative_data_compare = list(zip(*negative_data["edge_index"]))
negative_data_compare = set(list(map(lambda item: (int(item[0]),int(item[1])),negative_data_compare)))

simulated_edges_compare = list(zip(*simulated_edges_1))
simulated_edges_compare = set(list(map(lambda item: (int(item[0]),int(item[1])),simulated_edges_compare)))

In [20]:
precision = round(len(negative_data_compare.intersection(simulated_edges_compare))/len(negative_data_compare),2)
print("The amount of links that were in the data is the {} of the total".format(precision))

The amount of links that were in the data is the 0.5 of the total


In [21]:
nodes_class = list(range(len(nodes)))
n_links = 0
for item in simulated_edges_compare: 
    if item[0] in nodes_class: 
        n_links += 1
    else:
        print(item[0])
n_links

6742

In [22]:
print(f"We generated {len(simulated_edges_compare)} links, {n_links} of them are in the cut region ")
print(f"We cut a total of {len(negative_data_compare)} links")
print(f"From {n_links}, {int(precision*len(negative_data_compare))} of {len(negative_data_compare)} were found in the simulated links. ")

We generated 6742 links, 6742 of them are in the cut region 
We cut a total of 358 links
From 6742, 179 of 358 were found in the simulated links. 


### Link prediction with PageRank 

From the paper of _Alain Barrat_ Anxo recommended (_New Insights and Methods forPredicting Face-to-Face Contacts_), it can be checked the  _Hybrid Rooted PageRank_. We implement it in the following: 

*  With probability $\alpha$ jump to root node _r_.
*  With probability $1−\alpha$:
    *  Choose Network $N_{i}∈N$ with respect toprobability distribution _P_.
    *  If there exist no outgoing edges then :
    * Jump to root node _r_
    *  Else:
        From the current node c jump to a neighbornselected with a probability $w(c,n)∑c→dw(c,d)$, i. e.,proportional to the weight $w(c,n)$ of the $e(c,n)$

But we will include modifications on this analysis, as _Barrat et al_ use two networks in order to extract a single social network, while we are trying to deduce one from the other. We will implement PageRank on one of them and predict the links of the other one based on this quantity. When we are calculating link prediction, we calculate that the probability of the link is :
\begin{equation}
  p_{link(i,j)} =\dfrac{1}{1+e^{-(rank_{i}-rank_{j})}}
\end{equation}

Considering that the rank ordering is a some kind of classification of dominance of the node, according to the original paper. 

In [23]:
import random as rd
def HR_pagerank(alpha,G):
    N_rounds = 1000
    rank = [0]*len(G.nodes())
    for rounds in range(N_rounds):
        for node in range(len(nodes)):
            a = rd.uniform(0,1)
            site = list(G.nodes())[node]
            targets = list(G.nodes())
            targets.remove(site)
            if a > alpha:
                target = rd.choice(targets)
                if target in list(G.neighbors(site)):
                    c = rd.uniform(0,1)
                    weight_target = G[site][target]["peso"]
                    weight=nx.get_edge_attributes(G,'peso')
                    av_weights = 0
                    for n in list(G.neighbors(site)):
                        av_weights += weight[(site,n)]
                    av_weights /= len(list(G.neighbors(site)))
                    if c<((weight_target)/(av_weights)):
                        site = target
                        rank[site] +=1
    rank = [item/N_rounds for item in rank]
    return rank

def create_link(G,rank,chosen_edges):
    index_pair = rd.choice(chosen_edges)
    rd_pair = [rank[item] for item in index_pair]
    p_rank = 1/(1 + np.exp(-(rd_pair[0]-rd_pair[1])))
    if (rd.uniform(0,1) < p_rank) : 
        G.add_edge(index_pair[0],index_pair[1])
    return 

In [24]:
positive_rank = HR_pagerank(0.15,G_positive)
#positive_rank = nx.algorithms.pagerank(G_positive,0.15)
G_simulated = nx.DiGraph()
while len(G_simulated.edges())< final_edge_index_1.size()[1]:
    create_link(G_simulated,positive_rank,chosen_edges)

coincidences = to_dense_adj(negative_data["edge_index"]).squeeze()*torch.tensor(nx.adjacency_matrix(G_simulated).todense())
coin_rank_pos = coincidences.sum()

IndexError: list index out of range

### Randomly created network

We compare the results from the GNN and the PageRank with a randomly created network. 

In [ ]:
import random as rd 
coincidences_total = 0
for sim in range(10):
    chosen_edges_2 = chosen_edges.copy()
    G_random = nx.DiGraph()
    G_random.add_nodes_from(range(len(nodes)))
    for trial in range(final_edge_index_1.size()[1]):
        rd_sample = rd.choice(chosen_edges_2)
        G_random.add_edge(rd_sample[0],rd_sample[1]) 
        chosen_edges_2.remove(rd_sample)
    coincidences_random = len([(u,v) for (u,v) in G_random.edges() if G_negative.has_edge(u,v)])
    coincidences_total += coincidences_random
coin_random = coincidences_total/10

Proportions between the random network score and the amount of edges chosen divided by the total number of edges are similar, so the random network finds a correct proportion of the real links. 

In [ ]:
coin_random/len(neg_edges),final_edge_index_1.size()[1]/(len(nodes)*(len(nodes)-1)-len(pos_edges))

**Work to be done** 

1) Check the structural balance theory, computing global equilibria in both networks, in order to generate ensembles. 

2) Graph neural networks can also be used to predict labeling in edges, it could be used to proof structural balance theory from other perspective. 



In [ ]:
edges_saved_compare = list(G_random.edges())
edges_saved_compare = set(list(map(lambda item: (int(item[0]),int(item[1])),edges_saved_compare)))

simulated_edges_compare = list(zip(*simulated_edges_1))
simulated_edges_compare = set(list(map(lambda item: (int(item[0]),int(item[1])),simulated_edges_compare)))

In [ ]:
precision = round(len(edges_saved_compare.intersection(simulated_edges_compare))/len(simulated_edges_compare),2)
print("The amount of links that were in the data is the {} of the total".format(precision))